In [ ]:
import os
#os.environ['OPENAI_API_KEY'] = 'your-api-key'
#os.environ['YOUTUBE_API_KEY'] = 'your-api-key'

from dotenv import load_dotenv
load_dotenv(verbose=True)

---
### 01.Youtubeの動画URLリストを取得する。

 * apikeyの取得
   * https://developers.google.com/youtube/v3/getting-started?hl=ja
 * 参考:YouTube Data APIを使ってチャンネルに含まれる動画を取得する流れ
    * https://zenn.dev/yorifuji/articles/youtube-data-api


In [ ]:
# APIキーをセットします
import os
import pandas as pd
from apiclient.discovery import build
YOUTUBE_API_KEY = os.getenv('YOUTUBE_API_KEY')
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

In [ ]:
'''
チャンネルを検索する。
'''
channel_search_query = '山田五郎' #'山田五郎' #'高須幹弥' #'ゆる言語学ラジオ'

res = youtube.search().list(
    part='snippet',
    q=channel_search_query,
    type= 'channel', #'playlist',
    maxResults=10
).execute()

channel_df = (
    pd.DataFrame([
        {
            'channelTitle': x['snippet']['channelTitle'],
            'channelId' : x['snippet']['channelId'],
            # 'playlistTitle': x['snippet']['title'],
            # 'playlistId': x['id']['playlistId'],

        }
        for x in res['items']
    ])
)
display(channel_df)

In [ ]:
'''
検索したチャンネルから、オリジナルチャンネルのIDを調べるため、登録者数でソートする。
'''
records = []
for _, row in channel_df.iterrows():
    res = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=row['channelId'],
        maxResults=1
    ).execute()
    record = {
        'channelId': row['channelId'],
        'description' : res['items'][0]['snippet']['localized']['description'],
        'subscriberCount' : int(res['items'][0]['statistics']['subscriberCount']),
        'videoCount' : int(res['items'][0]['statistics']['videoCount']),
    }
    records.append(record)

channel_df=(
    channel_df
    .merge(pd.DataFrame(records), on='channelId')
    .sort_values('subscriberCount', ascending=False)
    .reset_index(drop=True)
)
display(channel_df)

In [ ]:
'''
オリジナルチャンネルの情報
 * 動画数は257本、登録者数は58万人らしい
'''
orginal_channel_row = channel_df.iloc[0]
orginal_channel_row

In [ ]:
'''
動画URLを取得する
'''
target_channelId = orginal_channel_row['channelId']
maxVideos = 500

records = []
nextPageToken=None
while len(records) < maxVideos:
    res = youtube.search().list(
        pageToken=nextPageToken,
        part='snippet',
        channelId=target_channelId,
        maxResults=50,
        type= 'video',
        # videoDuration = 'any' # 'short' 'medium' 'long'
    ).execute()

    nextPageToken = res.get('nextPageToken')
    records += [
        {
            'channelId': target_channelId,
            'videoId': x['id']['videoId'],
            'videoTitle': x['snippet']['title'],
            'videoDescription': x['snippet']['description'],
            'videoPublishTime': x['snippet']['publishedAt'],
        }
        for x in res['items']
    ]
    print(len(records), nextPageToken)
    if nextPageToken is None:
        break

In [ ]:
'''
動画URLを保存する
'''
video_df = (
    pd.DataFrame(records)
    .assign(videoPublishTime = lambda x: pd.to_datetime(x['videoPublishTime']))
    .sort_values('videoPublishTime', ascending=False)
    .reset_index(drop=True)
    .rename(columns={
        'channelId': 'channel_id',
        'videoId': 'video_id',
        'videoTitle': 'video_title',
        'videoDescription': 'video_description',
        'videoPublishTime': 'video_publish_time',
    })
)
video_df.to_csv('video_df.csv', index=False, encoding='utf-8-sig')
video_df

---
### 02.YoutubeのURLリストからスクリプトを取得する

In [ ]:
'''
01で作成したURLリストを読み込む
'''
video_df = pd.read_csv('video_df.csv')

In [ ]:
'''
スクリプトを順番に読み込む
* １ファイルあたり２秒程度
* add_video_info=Trueとする場合は、pytubeをインストールする必要がある
'''
from tqdm import tqdm
from langchain_community.document_loaders import YoutubeLoader

documents = []
for _, row in tqdm(video_df.iterrows(), total=video_df.shape[0]):
    url=f'https://www.youtube.com/watch?v={row["video_id"]}'
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True, language='ja')
    docs = loader.load()
    documents.extend(docs)


In [ ]:
#メタデータには以下のような情報が含まれています
documents[0].metadata

---
### 03.キーワード検索を作る

In [ ]:
import numpy as np
from typing import List
from langchain_core.documents import Document
from langchain_core.retrievers import  BaseRetriever

class MultiKeywordRetriever(BaseRetriever):
    docs: List[Document]
    k: int = 5

    def _get_relevant_documents(self, keywords: List[str]) -> List[Document]:
        keywords_counts = []
        for idx, doc in enumerate(self.docs):
            keywords_count = {k: doc.page_content.count(k) for k in keywords}
            keywords_count['min_count'] = min(keywords_count.values())  
            keywords_count['idx'] = idx
            keywords_counts.append(keywords_count)

        keywords_counts = [dic for dic in keywords_counts if dic['min_count'] > 0]
        keywords_counts = sorted(keywords_counts, key=lambda x: x['min_count'], reverse=True)
        keywords_counts = keywords_counts[:self.k]

        relevant_documents = [self.docs[dic['idx']] for dic in keywords_counts]
        return relevant_documents

In [ ]:
'''
ドガとマネに言及している動画
'''
keywords = ['ドガ','マネ']
retriever = MultiKeywordRetriever(docs=documents,k=20)
retriever.invoke(keywords)

In [ ]:
'''
ドラえもんに言及している動画
'''
keywords = ['ドラえもん']
retriever = MultiKeywordRetriever(docs=documents,k=20)
retriever.invoke(keywords)

---
### 04.クエリからキーワードを生成する

In [ ]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Search(BaseModel):
    keywords: List[str] = Field(description="キーワード検索で使用するキーワード")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """あなたはユーザーの質問から固有名詞のキーワードを抽出するエキスパートです。
例えば「ドガとマネの絵画について教えて下さい」という質問が来たら、["ドガ", "マネ"]というキーワードを抽出します。"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        # MessagesPlaceholder("examples", optional=True),
        ("human", "{question}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm = llm.with_structured_output(Search)
search_chain = (    
    {"question": RunnablePassthrough()}
    | prompt
    | structured_llm
)
res = search_chain.invoke('ドラえもんについて何が言及されていますか？')
res

In [ ]:
ret_docs = retriever.invoke(res.keywords)
ret_docs

In [ ]:
from pprint import pprint
for d in ret_docs:
    pprint(d.metadata)

---
### 05.文書から情報を抽出する

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

prompt_template = """以下の質問についてドキュメントをもとに回答してください。
ただしドキュメントはYoutubeの音声を文字起こししたもののため、不明瞭な文章の可能性があります。
そのため、回答には注意してください。

ドキュメント:{texts}

質問:{question}

回答:"""

prompt = PromptTemplate(
    template = prompt_template,
    input_variables=["texts", "question"],
)
answer_chain = prompt | llm

In [ ]:
res = answer_chain.invoke({
    'texts':[doc.page_content for doc in ret_docs],
    'question':'ドラえもんについて何が言及されていますか？'
})
res

In [ ]:
print(res.content)

---
### 06.これまでのをまとめて質問

In [ ]:
question = '「猫」についての言及をまとめて？' 
res = search_chain.invoke(question)
print(res)
ret_docs = retriever.invoke(res.keywords)
print(ret_docs)
res = answer_chain.invoke({
    'texts':ret_docs[0].page_content,
    'question':question,
})
print(res.content)